# Quarter-Mile Race from Standing Start: Pick Your Cars
This notebook simulates a standing-start quarter-mile race (402.336 m) between two cars.
## How to use
- Run all cells once; an interactive form appears automatically.
- Select a preset for **Car 1** and **Car 2** from the dropdowns (each preset has a
  meaningful name independent of its powertrain).
- Tweak mass, tire compound / width, and (for ICE cars) launch RPM, shift RPM, and
  shift time using the sliders.
- Press **▶  Run Race** to re-run the simulation and update all charts.
## What is configurable per car
- Powertrain type (`ICE`, `BEV`), drivetrain (`RWD` or `AWD`), and tires are set by the
  preset; you can further adjust mass and tire spec in the form.
- ICE cars expose launch RPM, shift RPM, and manual shift time.
- The simulation uses forward-Euler integration with traction-limited launch, aero drag,
  and rolling resistance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import all simulation logic from the reusable module.
# quarter_mile_sim.py lives alongside this notebook, which makes it
# importable in CPython, JupyterLab, and JupyterLite (Pyodide / WASM).
from quarter_mile_sim import (
    make_car,
    simulate_quarter_mile,
    interp_curve,
    QUARTER_MILE_M,
    DEFAULT_DT,
)

# Notebook-level aliases kept for readability in the cells below
quarter_mile_m = QUARTER_MILE_M
dt = DEFAULT_DT


In [ ]:
# ── Car database ─────────────────────────────────────────────────────────────
# Car names describe character/body style, not powertrain config.
# Each entry follows the Car → Powertrain → Motor / Gearbox schema used by make_car().

CAR_DATABASE = {
    "Sport Coupe": {
        "name": "Sport Coupe",
        "vehicle": {
            "mass": 1500, "CdA": 0.66, "wheel_radius_m": 0.335,
            "rolling_resistance": 0.015,
            "tire": {"width_mm": 300, "compound": "summer", "base_mu": 1.10},
        },
        "powertrain": {
            "type": "ICE", "driving_axles": "RWD",
            "efficiency": {"engine": 1.0, "driveline": 0.90},
            "motors": [{"name": "ICE Engine", "min_rpm": 900, "max_rpm": 7200,
                "torque_curve_rpm_nm": [[1000,380],[2000,570],[3000,860],[4000,850],
                    [5000,850],[6000,800],[7000,690],[7300,650],[8000,100]]}],
            "gearbox": {"type": "manual", "gear_ratios": [3.10,2.10,1.55,1.22,1.00,0.82],
                "final_drive": 3.73, "launch_rpm": 2800, "shift_rpm": 6900, "shift_time_s": 0.30},
        },
    },
    "Electric Sedan": {
        "name": "Electric Sedan",
        "vehicle": {
            "mass": 2400, "CdA": 0.68, "wheel_radius_m": 0.350,
            "rolling_resistance": 0.016,
            "tire": {"width_mm": 300, "compound": "summer", "base_mu": 1.05},
        },
        "powertrain": {
            "type": "BEV", "driving_axles": "AWD",
            "efficiency": {"motor": 1.0, "inverter": 0.96},
            "motors": [{"name": "Combined eMotors", "min_rpm": 0, "max_rpm": 16000,
                "torque_curve_rpm_nm": [[0,850],[6000,850],[7000,810],[8000,660],
                    [10000,520],[12000,420],[14000,340],[16000,290]]}],
            "gearbox": {"type": "single_speed", "ratio": 9.0},
        },
    },
    "Muscle Car": {
        "name": "Muscle Car",
        "vehicle": {
            "mass": 1900, "CdA": 0.72, "wheel_radius_m": 0.345,
            "rolling_resistance": 0.016,
            "tire": {"width_mm": 315, "compound": "summer", "base_mu": 1.08},
        },
        "powertrain": {
            "type": "ICE", "driving_axles": "RWD",
            "efficiency": {"engine": 1.0, "driveline": 0.88},
            "motors": [{"name": "V8 Engine", "min_rpm": 700, "max_rpm": 6500,
                "torque_curve_rpm_nm": [[800,550],[1500,700],[2500,820],[3500,850],
                    [4500,830],[5500,760],[6500,620],[7000,400]]}],
            "gearbox": {"type": "auto",
                "gear_ratios": [4.17,2.34,1.52,1.14,0.87,0.69,0.55,0.46],
                "final_drive": 3.31, "launch_rpm": 1800, "shift_rpm": 6200, "shift_time_s": 0.10},
        },
    },
    "Electric Hypercar": {
        "name": "Electric Hypercar",
        "vehicle": {
            "mass": 1350, "CdA": 0.55, "wheel_radius_m": 0.340,
            "rolling_resistance": 0.014,
            "tire": {"width_mm": 325, "compound": "track", "base_mu": 1.15},
        },
        "powertrain": {
            "type": "BEV", "driving_axles": "AWD",
            "efficiency": {"motor": 1.0, "inverter": 0.97},
            "motors": [{"name": "Dual Motors", "min_rpm": 0, "max_rpm": 20000,
                "torque_curve_rpm_nm": [[0,1250],[5000,1250],[8000,1050],[11000,780],
                    [14000,580],[17000,420],[20000,300]]}],
            "gearbox": {"type": "single_speed", "ratio": 10.5},
        },
    },
    "Performance Wagon": {
        "name": "Performance Wagon",
        "vehicle": {
            "mass": 1780, "CdA": 0.70, "wheel_radius_m": 0.340,
            "rolling_resistance": 0.015,
            "tire": {"width_mm": 295, "compound": "summer", "base_mu": 1.07},
        },
        "powertrain": {
            "type": "ICE", "driving_axles": "AWD",
            "efficiency": {"engine": 1.0, "driveline": 0.88},
            "motors": [{"name": "Turbocharged Engine", "min_rpm": 800, "max_rpm": 6800,
                "torque_curve_rpm_nm": [[1000,420],[1800,600],[2500,680],[3500,670],
                    [4500,650],[5500,610],[6500,540],[7000,380]]}],
            "gearbox": {"type": "auto",
                "gear_ratios": [3.91,2.29,1.55,1.16,0.86,0.73],
                "final_drive": 3.55, "launch_rpm": 2200, "shift_rpm": 6500, "shift_time_s": 0.08},
        },
    },
}

In [ ]:
import copy
import ipywidgets as widgets
from IPython.display import display

_TIRE_COMPOUNDS = ["all_season", "summer", "track", "drag_radial"]
_DB_KEYS = list(CAR_DATABASE.keys())


def _make_car_form(slot_label, default_key):
    """Build ipywidgets form for one car slot; returns (box_widget, get_spec_fn)."""
    preset_dd = widgets.Dropdown(
        options=_DB_KEYS, value=default_key,
        description=f"{slot_label}:",
        style={"description_width": "50px"},
        layout=widgets.Layout(width="210px"),
    )
    pt_label = widgets.Label(layout=widgets.Layout(margin='2px 0 0 8px'))
    mass_txt  = widgets.FloatText(description="Mass (kg):",
        style={"description_width": "80px"}, layout=widgets.Layout(width="220px"))
    cmpd_dd   = widgets.Dropdown(options=_TIRE_COMPOUNDS, description="Tire cmpd:",
        style={"description_width": "80px"}, layout=widgets.Layout(width="220px"))
    tirw_sl   = widgets.IntSlider(min=185, max=345, step=5, description="Width (mm):",
        style={"description_width": "80px"}, layout=widgets.Layout(width="310px"))
    # ICE-only widgets
    lrpm_sl   = widgets.IntSlider(min=500,  max=4000, step=100, description="Launch RPM:",
        style={"description_width": "90px"}, layout=widgets.Layout(width="310px"))
    srpm_sl   = widgets.IntSlider(min=3000, max=8500, step=100, description="Shift RPM:",
        style={"description_width": "90px"}, layout=widgets.Layout(width="310px"))
    stim_sl   = widgets.FloatSlider(min=0.05, max=0.60, step=0.05, readout_format='.2f',
        description="Shift time s:",
        style={"description_width": "90px"}, layout=widgets.Layout(width="310px"))
    ice_vbox  = widgets.VBox([lrpm_sl, srpm_sl, stim_sl])

    def _populate(key):
        spec = CAR_DATABASE[key]
        pt   = spec["powertrain"]
        pt_label.value = f"  {pt['type'].upper()} · {pt['driving_axles'].upper()}"
        mass_txt.value = spec["vehicle"]["mass"]
        cmpd_dd.value  = spec["vehicle"]["tire"].get("compound", "summer")
        tirw_sl.value  = spec["vehicle"]["tire"]["width_mm"]
        is_ice = pt["type"].upper() == "ICE"
        ice_vbox.layout.display = "" if is_ice else "none"
        if is_ice:
            gb = pt["gearbox"]; m0 = pt.get("motors", [{}])[0]
            lrpm_sl.value = gb.get("launch_rpm", m0.get("min_rpm", 900))
            srpm_sl.value = gb.get("shift_rpm",  m0.get("max_rpm", 7000))
            stim_sl.value = gb.get("shift_time_s", 0.30)

    _populate(default_key)
    preset_dd.observe(lambda ch: _populate(ch["new"]), names="value")

    def get_spec():
        key  = preset_dd.value
        spec = copy.deepcopy(CAR_DATABASE[key])
        spec["vehicle"]["mass"]              = mass_txt.value
        spec["vehicle"]["tire"]["compound"]  = cmpd_dd.value
        spec["vehicle"]["tire"]["width_mm"]  = tirw_sl.value
        if spec["powertrain"]["type"].upper() == "ICE":
            gb = spec["powertrain"]["gearbox"]
            gb["launch_rpm"]  = lrpm_sl.value
            gb["shift_rpm"]   = srpm_sl.value
            gb["shift_time_s"] = stim_sl.value
        return key, spec

    box = widgets.VBox(
        [widgets.HBox([preset_dd, pt_label]), mass_txt, cmpd_dd, tirw_sl, ice_vbox],
        layout=widgets.Layout(border="1px solid #ccc", padding="8px",
                              margin="4px", min_width="340px"),
    )
    return box, get_spec


form1, get_spec1 = _make_car_form("Car 1", _DB_KEYS[0])
form2, get_spec2 = _make_car_form("Car 2", _DB_KEYS[1])

run_btn = widgets.Button(description="▶  Run Race", button_style="success",
                         layout=widgets.Layout(width="150px", margin="8px 0"))
out = widgets.Output()


def _run_race(_=None):
    name1, spec1 = get_spec1()
    name2, spec2 = get_spec2()
    car_specs = {name1: spec1, name2: spec2}
    cars    = {n: make_car(n, s) for n, s in car_specs.items()}
    results = {n: simulate_quarter_mile(c, dt=dt) for n, c in cars.items()}
    with out:
        out.clear_output(wait=True)
        # ── Summary ──────────────────────────────────────────────────────────
        for name, r in results.items():
            car = cars[name]
            line = (f"{name} | {car['powertrain_type']} | {car['drivetrain']} |"
                    f" {car['tire_width_mm']:.0f}mm {car['tire_compound']}")
            if car["powertrain_type"] == "ICE":
                line += (f" | {car['ice']['gearbox_type']}"
                         f" {len(car['ice']['gear_ratios'])}spd"
                         f" | shifts={r['shift_count']}")
            print(line)
            print(f"  mass={car['mass']:.0f} kg  →  ET={r['elapsed_time']:.2f} s,"
                  f" trap={r['trap_speed']*3.6:.1f} km/h")
        winner = min(results.items(), key=lambda kv: kv[1]["elapsed_time"])[0]
        print(f"\n🏁 Winner over quarter mile: {winner}")
        # ── Plot 1: Acceleration and wheel torque vs speed ───────────────────
        fig, ax1 = plt.subplots(figsize=(10, 4))
        ax2 = ax1.twinx()
        c1, c2 = "tab:blue", "tab:red"
        ax1.set_xlabel("Speed (km/h)"); ax1.set_ylabel("Acceleration (m/s²)", color=c1)
        ax2.set_ylabel("Wheel Torque (Nm)", color=c2)
        ax1.tick_params(axis="y", labelcolor=c1); ax2.tick_params(axis="y", labelcolor=c2)
        for name, r in results.items():
            spd = r["speed"] * 3.6; order = np.argsort(spd)
            ax1.plot(spd[order], r["accel"][order], label=name, linewidth=2)
            ax2.plot(spd[order], r["wheel_torque"][order],
                     label=f"{name} (torque)", linestyle="--", linewidth=2)
        ax1.set_title("Acceleration and Wheel Torque vs Speed")
        ax1.grid(True, alpha=0.3); ax1.legend(loc="upper left"); ax2.legend(loc="upper right")
        plt.tight_layout(); plt.show()
        # ── Plot 2: Power curves ─────────────────────────────────────────────
        fig, ax = plt.subplots(figsize=(10, 4))
        for name, car in cars.items():
            if car["powertrain_type"] == "ICE":
                rpm_r = np.linspace(0, car["ice"]["redline_rpm"], 150)
                tq    = np.array([interp_curve(car["ice"]["torque_curve_rpm_nm"], r) for r in rpm_r])
            else:
                rpm_r = np.linspace(0, car["motor"]["max_rpm"], 150)
                tq    = np.array([interp_curve(car["motor"]["torque_curve_rpm_nm"], r) for r in rpm_r])
            ax.plot(rpm_r, tq * rpm_r / 7745, label=name, linewidth=2)
        ax.set_xlabel("RPM"); ax.set_ylabel("Power (HP)")
        ax.set_title("Power Curve vs RPM"); ax.grid(True, alpha=0.3); ax.legend()
        plt.tight_layout(); plt.show()
        # ── Plot 3: Distance and speed vs time ───────────────────────────────
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        for name, r in results.items():
            axes[0].plot(r["time"], r["distance"], label=name)
            axes[1].plot(r["time"], r["speed"] * 3.6, label=name)
        axes[0].axhline(quarter_mile_m, linestyle="--", linewidth=1,
                        color="k", alpha=0.7, label="Quarter mile")
        axes[0].set_title("Distance vs Time"); axes[0].set_xlabel("Time (s)")
        axes[0].set_ylabel("Distance (m)"); axes[0].grid(True, alpha=0.3); axes[0].legend()
        axes[1].set_title("Speed vs Time"); axes[1].set_xlabel("Time (s)")
        axes[1].set_ylabel("Speed (km/h)"); axes[1].grid(True, alpha=0.3); axes[1].legend()
        plt.tight_layout(); plt.show()


run_btn.on_click(_run_race)
display(widgets.HBox([form1, form2]))
display(run_btn)
display(out)
_run_race()  # auto-run on cell execution

## Notes
- Select presets from the dropdown; car names describe character (e.g. *Sport Coupe*,
  *Muscle Car*), not drivetrain type.
- Adjust mass, tire compound/width, and (for ICE) launch/shift RPM in the form, then
  press **▶  Run Race**.
- `powertrain.driving_axles` (`RWD`/`AWD`) and `powertrain.type` (`ICE`/`BEV`) are
  set by the preset and shown as a read-only label next to the preset dropdown.
- Manual shift (`shift_time_s`) introduces a zero-wheel-power window during each shift.
- Wheel power includes efficiency losses and is always lower than source power.
- To add a custom preset, extend `CAR_DATABASE` in the cell above the form.